# SI 618 Final Project Report

Team member:
|  Name  |  uniqname |   UMID 
|:------|:-------|:-------|
| Yung-Yi (Cerise) Huang | superb |   | 
| Ming-Jan (Randy) Pan | mjpan  |  91936606 | 

## 1. Introduction

### 1.1 Background and Rationale

### 1.2 Research Questions

Some questions we wish to answer are:
1. How does physical activity, dietary habits, and weight status surveillance results change over time for each state in the US? 
2. Does having higher average SNAP benefits per person make for better dietary habits?
3. Does having higher state walkability increase overall physical activity?


### 1.3 What is SNAP

### 1.4 What is walkability

### 1.5 Theoretical relationship between research variables

## 2. Data Collection and Preprocessing

### 2.1 Datasets

In this project, we utilized 4 datasets to answer our questions as well as make visualizations. The datasets and their acquisition methods are listed below:

<style>
    /* .heatMap {
        width: 70%;
        text-align: center;
    } */
    .datatb th {
        background: grey;
        word-wrap: break-word;
        text-align: center;
    }
    .datatb tr:nth-child(1) { background: yellow; opacity: 0.5; }
    /* .datatb tr:nth-child(2) { background: orange; }
    .datatb tr:nth-child(3) { background: green; } */
</style>

<div class="datatb">

| Dataset  | Source | Description | Shape (rows, cols) | Missing data % | 
| -- | -- | -- | -- | -- | 
| **[Primary dataset - Nutrition, Physical Activity, and Obesity - Youth Risk Behavior Surveillance System](https://data.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Youth-Risk/vba9-s8jp)** | CDC<sup>1</sup> | This dataset contains surveillance of adolescents’ (9th-12th grade) physical activity, diet, and weight statuses aggregated on the state level and stratified by gender, race/ethnicity, and grade from 2001 to 2019. This dataset is directly downloaded from CDC website | (39760, 25)	 | 19.47 |  
| [Demographics and the Economy Indicators Supplemental Nutrition Assistance Program (SNAP)](https://www.kff.org/state-category/demographics-and-the-economy/supplemental-nutrition-assistance-program/) | KFF<sup>2</sup> | This dataset contains the statistics on the SNAP program (yearly average enrollment, total benefits, and benefits per person). The data is crawled from the Kaiser Family Foundation website using the selenium library<sup>3</sup>.  | (1134, 5) |  0.035 | 
| [City and Neighborhood Walkability Rankings](https://www.walkscore.com/cities-and-neighborhoods/states/) | Walk Score | This dataset contains walkability and bike score of each state. The original data contains walkability score at the county level, which will be aggregated to state level in our analysis process. | (2500, 7) | 8.51 |
| [Cartographic Boundary Files - Shapefile (2022), County level, resolution 1:500,000](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html) | US Census Bureau | This dataset contains the cartographic geometry for all US counties at 1:500,000 resolution to plot choropleth maps. This dataset is processed using the geopandas package. This dataset is obtained directly from the census bureau |(56, 5)  | 0.0 |

</div>

<sup>1</sup> CDC: Centers for Disease Control and Prevention\
<sup>2</sup> KFF: Kaiser Family Foundation\
<sup>3</sup> The code for crawling the data is in the `get_data.py` file within the `data` folder.

### 2.2 Data Preprocessing

#### 2.2.1 Primary dataset - Nutrition, Physical Activity, and Obesity - Youth Risk Behavior Surveillance System

1. We dropped all the Guam data because they are not states in the US.
2. We also dropped the "Data_Value_Unit", "Total", "Data_Value_Type", "GeoLocation", "YearEnd","Datasource" columns because they either contain too many missing values, redundant data, or are not useful for our analysis.

## 3. Data Analysis and Visualization

In this section, we present our findings.

### 3.1 Data Preparation

In [4]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import geopandas as gpd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from jenkspy import JenksNaturalBreaks
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from tqdm import tqdm
import imageio
import os

import requests
import warnings
warnings.filterwarnings("ignore")

In [5]:
snap_df = pd.read_csv("../data/SNAP_data_raw.csv")

snap_df["Average Monthly SNAP Benefits per Participant"] = (
    snap_df["Average Monthly SNAP Benefits per Participant"]
    .str.removeprefix("$")
    .replace("N/A", np.nan)
    .astype("float32")
)

snap_df["Average Monthly SNAP Participants"] = (
    snap_df["Average Monthly SNAP Participants"]
    .str.removesuffix(" 1")
    .str.replace(",", "")
    .replace("N/A", np.nan)
    .astype("float32")
)

snap_df["Total Benefits"] = (
    snap_df["Total Benefits"]
    .str.removeprefix("$")
    .str.replace(",", "")
    .str.removesuffix(" 1")
    .replace("N/A", np.nan)
    .astype("int64")
)

snap_df["year"] = snap_df["year"].astype("int16")

snap_df.drop("Unnamed: 0", axis=1, inplace=True)

# geometry shape file - state level
sgdf = gpd.read_file("../data/cb_2022_us_state_500k/cb_2022_us_state_500k.shp")
sgdf = sgdf.drop(['ALAND', 'AWATER','GEOID', 'AFFGEOID', 'STATENS'], axis=1)

In [6]:
df_nutri = pd.read_csv("../data/nutrition_activity.csv")

# YearStart and YearEnd contain the same data
assert (df_nutri["YearEnd"] != df_nutri["YearStart"]).sum() == 0

# We give the columns dropped and reasons for dropping
# 1. Data_Value_Unit: Contains all null data
# 2. Total
# 3. Data_Value_Type
# 4. Geolocation: Shape file will be merged externally for choropleth mapping
# 5. YearEnd: Contains the same data as "YearStart"
# 6. Datasource: Contains the same vale: "Youth Risk Behavior Surveillance System"
df_nutri.drop(
    [
        "Data_Value_Unit",
        "Total",
        "Data_Value_Type",
        "GeoLocation",
        "YearEnd",
        "Datasource",
    ],
    axis=1,
    inplace=True,
)
df_nutri = df_nutri.query("LocationDesc != 'Guam'")

In [7]:
df_walk = pd.read_csv("../data/walkability.csv")

In [9]:
# get the shape and missing data percentage of all dataframes 
pd.DataFrame(
    {
        "Nutrition data (CDC)": [
            df_nutri.shape,
            (df_nutri.isna().sum().sum() / (df_nutri.shape[0] * df_nutri.shape[1])) * 100,
        ],
        "City Walkability and Bikability": [
            df_walk.shape,
            (df_walk.isna().sum().sum() / (df_walk.shape[0] * df_walk.shape[1])) * 100,
        ],
        "SNAP Benefits statistics": [
            snap_df.shape,
            (snap_df.isna().sum().sum() / (snap_df.shape[0] * snap_df.shape[1])) * 100,
        ],
        "Cartographic Boundary Files - State Level": [
            sgdf.shape,
            (sgdf.isna().sum().sum() / (sgdf.shape[0] * sgdf.shape[1])) * 100,
        ],
    }
).T.rename(columns={0: "Shape (rows, cols)", 1: "% missing data"})

,"Shape (rows, cols)",% missing data
Nutrition data (CDC),"(39760, 25)",19.469215
City Walkability and Bikability,"(2500, 7)",8.508571
SNAP Benefits statistics,"(1134, 5)",0.035273
Cartographic Boundary Files - State Level,"(56, 5)",0.0


### 3.2 Question 1

Unique question in the dataset:

|    | ClassID   | Class                   | QuestionID   | Question                                                                                                                    |
|---:|:----------|:------------------------|:-------------|:----------------------------------------------------------------------------------------------------------------------------|
|  0 | FV        | Fruits and Vegetables   | Q020         | Percent of students in grades 9-12 who consume fruit less than 1 time daily                                                 |
|  1 | FV        | Fruits and Vegetables   | Q021         | Percent of students in grades 9-12 who consume vegetables less than 1 time daily                                            |
|  2 | OWS       | Obesity / Weight Status | Q038         | Percent of students in grades 9-12 who have obesity                                                                         |
|  3 | OWS       | Obesity / Weight Status | Q039         | Percent of students in grades 9-12 who have an overweight classification                                                    |
|  4 | PA        | Physical Activity       | Q048         | Percent of students in grades 9-12 who achieve 1 hour or more of moderate-and/or vigorous-intensity physical activity daily |
|  5 | PA        | Physical Activity       | Q049         | Percent of students in grades 9-12 who participate in daily physical education                                              |
|  6 | SD        | Sugar Drinks            | Q058         | Percent of students in grades 9-12 who drank regular soda/pop at least one time per day                                     |
|  7 | TV        | Television Viewing      | Q059         | Percent of students in grades 9-12 watching 3 or more hours of television each school day                                   |

(table generated with code:

```python
print(df_nutri.groupby(["ClassID", "QuestionID"])["Question"]
      .unique()
      .reset_index()
      .explode("Question")
      .to_markdown())
```

)



In [ ]:
vegetable_trend = (
    pd.pivot_table(
        df_nutri.query("ClassID == 'FV'"),
        index="YearStart",
        columns="QuestionID",
        values="Data_Value",
    )
    .reset_index()
    .assign(YearStart=lambda x: pd.to_datetime(x["YearStart"], format="%Y"))
)

register_matplotlib_converters()  # This is needed for pandas datetime

# Plotting
ax = vegetable_trend.plot.line(x="YearStart")
ax.set_title(
    "Historical trend of students consuming\nvegetables or fruits less than 1 time daily"
)
xticks = pd.date_range(
    start=vegetable_trend["YearStart"].min(), end="2020-01-01", freq="2Y"
)
ax.set_xticks(xticks)
ax.set_xticklabels([x.strftime("%Y") for x in xticks], rotation=45)
L = plt.legend()
L.get_texts()[0].set_text("Fruit (Q020)")
L.get_texts()[1].set_text("Vegetables (Q021)")

In [ ]:
obesity_trend = (
    pd.pivot_table(
        df_nutri.query("ClassID == 'OWS' & Stratification1 == 'Total'"),
        index="YearStart",
        columns="QuestionID",
        values="Data_Value",
    )
    .reset_index()
    .assign(YearStart=lambda x: pd.to_datetime(x["YearStart"], format="%Y"))
)

register_matplotlib_converters()  # This is needed for pandas datetime

# Plotting
ax = obesity_trend.plot.line(x="YearStart")
ax.set_title("Historical trend of percent of students who are overweight or obese")
xticks = pd.date_range(
    start=obesity_trend["YearStart"].min(), end="2020-01-01", freq="2Y"
)
ax.set_xticks(xticks)
ax.set_xticklabels([x.strftime("%Y") for x in xticks], rotation=45)
L = plt.legend()
L.get_texts()[0].set_text("Obesity(Q038)")
L.get_texts()[1].set_text("Overweight (Q039)")

In [ ]:
soda_trend = (
    pd.pivot_table(
        df_nutri.query("ClassID == 'SD'"),
        index="YearStart",
        columns="QuestionID",
        values="Data_Value",
    )
    .reset_index()
    .assign(YearStart=lambda x: pd.to_datetime(x["YearStart"], format="%Y"))
)

register_matplotlib_converters()  # This is needed for pandas datetime

# Plotting
ax = soda_trend.plot.line(x="YearStart")
ax.set_title("Historical trend of students' soda consumption status")
xticks = pd.date_range(start=soda_trend["YearStart"].min(), end="2020-01-01", freq="2Y")
ax.set_xticks(xticks)
ax.set_xticklabels([x.strftime("%Y") for x in xticks], rotation=45)
L = plt.legend()
L.get_texts()[0].set_text("Soda consumption (Q058)")

### 3.3 Question 2 

Does having higher average SNAP benefits per person make for better dietary habits? 

### 3.4 Question 3

Does having higher state walkability increase overall physical activity?

## 4. Discussion and Conclusion

### 4.1 Discussion

### 4.2 Limitations

- Data availability: we could not obtain county level datasets to conduct our analysis. 

### 4.3 Conclusion